In [ ]:
import numpy as np

In [ ]:
#FUNCION IM2COL EN PYTHON (FUNCIONES PARA PODER HACER LA CONVOLUCION MUCHO MÁS RAPIDO)
#FUNCION PARA UNA SOLA IMAGEN DE 2 DIMENSIONES
def vec2D(A):
  blocksize = A.shape
  stepsize = 1
  # Parameters
  M,N = A.shape
  col_extent = N - blocksize[1] + 1
  row_extent = M - blocksize[0] + 1
    
    # Get Starting block indices
  start_idx = (np.arange(blocksize[0])[:,None]*N + np.arange(blocksize[1])).T
    
    
    # Get offsetted indices across the height and width of input array
  offset_idx = (np.arange(row_extent)[:,None]*N + np.arange(col_extent)).T
    
    # Get all actual indices & index into input array for final output
  return np.take(A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

#FALTA DEFINIR FUNCION PARA VECTORIZAR ARRAYS 3D

def im2Dcol(A, blocksize, stride, stepsize=1):
  # Parameters
  M,N = A.shape
  col_extent = int((N - blocksize[1])/stride + 1)
  row_extent = int((M - blocksize[0])/stride + 1)
    
    # Get Starting block indices
  start_idx = (np.arange(blocksize[0])[:,None]*N + np.arange(blocksize[1])).T
    
    # Get offsetted indices across the height and width of input array
  offset_idx = ((np.arange(row_extent)[:,None]*N + np.arange(col_extent))*stride).T
    
    # Get all actual indices & index into input array for final output
  return np.take(A,start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize])

#FUNCION DE EXPANSION PARA UN CONJUNTO DE IMAGENES DE C CANALES
def imNDcol(A, blocksize, stride, maxpool = False, stepsize=1):
  m, n_C, n_H, n_W = A.shape
  #blocksize = [3,2,2]
  col_extent = int((n_W - blocksize[2])/stride + 1)
  row_extent = int((n_H - blocksize[1])/stride + 1)
      
  # Get Starting block indices
  start_idx = (np.arange(blocksize[1])[:,None]*n_W + np.arange(blocksize[2])).T
  start_idx = np.broadcast_to(start_idx, (1,n_C,blocksize[1], blocksize[2]))
  idx_s = np.arange(0,n_W*n_W*n_C,n_W*n_W)[:, None]
  idx_s = idx_s[None, :]
  idx_s = idx_s[:,:,np.newaxis]
  start_idx = start_idx + idx_s 

  # Get offsetted indices across the height and width of input array
  offset_idx = ((np.arange(row_extent)[:,None]*n_W + np.arange(col_extent))*stride).T
  h_o, w_o = offset_idx.shape
  offset_idx = np.broadcast_to(offset_idx, (m, 1, h_o, w_o))
  idx_o = np.arange(0,n_W*n_H*n_C*m, n_W*n_H*n_C)[:, None]
  idx_o = idx_o[:,:,np.newaxis,np.newaxis]
  offset_idx = offset_idx + idx_o
  
  # Get all actual indices & index into input array for final output
  indices = start_idx.ravel()[:,None] + offset_idx.ravel()[::stepsize]

  if maxpool:
    return np.take(A, indices), indices
  else:
    return np.take(A, indices)

In [ ]:
# FUNCIONES PARA LA CONVOLUCIÓN PERO SIN BUCLES FOR(Se trata de usar en lo mínimo posible el ciclo for)
# imgIn => (#ejemplos)x(#canales)x(ancho)x(altura)
# Filters => (#filtros)x(#canales)x(ancho)x(altura)
def conv4D(imgIn, filters, padding, stride):
  #ESTA FUNCION BOTA AL CONVOLUCIÓN PERO NO CON LA FORMA QUE DEBERÍA BOTAR (FALTA MODIFICAR)
  #PERO EL CÁLCULO ES CORRECTO
  m, n_C, n_H, n_W = imgIn.shape
  n_f, f_C, f_H, f_W = filters.shape

  B = imNDcol(imgIn, [f_C, f_H, f_W], stride = stride)
  F = imNDcol(filters, [f_C, f_H, f_W], stride = stride)
  #Convolución
  CONV = B.T @ F

  return CONV

#FALTA MODIFICAR (EL MAXPOOLING SE HACE A CADA CANAL DE LA IMAGEN DE ENTRADA)
def maxpooling(X, filsize, padding, stride):
  m, n_C, n_H, n_W = X.shape
  blocksize = [n_C, filsize, filsize]

  #Se obtiene la matriz de expansión y los indices que ayudarán a la derivada
  B, indices= imNDcol(X, blocksize, stride, maxpool = True)

  #Ciclo for para hallar los máximos
  #Entre todas las filas de B están distribuidas los n_C canales
  #Bpool contiene los maximos 
  Bpool = np.zeros((n_C, B.shape[1]))
  Bmax = np.zeros(indices.shape)
  k = 0
  step = filsize**2
  for i in range(n_C):
    Bpool[i] = np.amax(B[i+k:i+step+k], axis = 0, keepdims = True)
    ind_max = np.argmax(B[i+k:i+step+k], axis = 0)[None, :]
    np.put_along_axis(Bmax[i+k:i+step+k], ind_max, 1, axis = 0)
    k = k + step - 1

  return Bpool, Bmax